# Exploring the Point Generators

In this notebook we explore the three different point generator classes.

1. The *PointGenerator* class is the base class the other two inherit from. It can be used for Complete Intersection Calabi-Yau (CICY) manifolds with a single hypersurface, e.g. the quintic, bicubic, p1p3, the tetraquadric, or a torus, some K3s ...
2. There is the *CICYPointGenerator* which works for any CICY.
3. There is the *ToricPointGenerator* which can generate points on any CY coming from the Kreuzer-Skarke list. NOTE: as of v0.0.1 there are still some issues with finding the correct integration weights from generalizing a theorem by Shiffman and Zelditch.

The Fermat quintic manifold can be implemented in each of the three PointGenerator classes and will be used as an example.

Recall that the Fermat quintic is given by the following polynomial:

$$
Q = \sum_i^5 z_i^5
$$

in a $\mathbb{P}^4$ ambient space.

In [1]:
import numpy as np
import os as os
import pickle as pickle

## PointGenerator

First we load the PointGenerator from the cymetric package.

In [2]:
from cymetric.pointgen.pointgen import PointGenerator

In general all routines and classes of the cymetric package will have a brief description of their functionality and their arguments. You can access it with the help function.

In [3]:
help(PointGenerator)

Help on class PointGenerator in module cymetric.pointgen.pointgen:

class PointGenerator(builtins.object)
 |  PointGenerator(monomials, coefficients, kmoduli, ambient, vol_j_norm=1, verbose=2, backend='multiprocessing')
 |  
 |  The PointGenerator class.
 |  
 |  The numerics are entirely done in numpy; sympy is used for taking 
 |  (implicit) derivatives.
 |  
 |  Use this one if you want to generate points and data on a CY given by
 |  one hypersurface.
 |  
 |  All other PointGenerators inherit from this class.
 |  
 |  Example:
 |      We consider the Fermat quintic given by
 |  
 |      .. math::
 |  
 |          Q(z) = z_1^5 + z_2^5 + z_3^5 + z_4^5 + z_5^5
 |  
 |      and set it up with:
 |  
 |      >>> import numpy as np
 |      >>> from cymetric.pointgen.pointgen import PointGenerator
 |      >>> monomials = 5*np.eye(5, dtype=np.int)
 |      >>> coefficients = np.ones(5)
 |      >>> kmoduli = np.ones(1)
 |      >>> ambient = np.array([4])
 |      >>> pg = PointGenerator(monom

In the next step we define the five defining monomials, the coefficients in front of each monomial, the single Kähler moduli, and the ambient space projective factor(s):

In [4]:
monomials = 5*np.eye(5, dtype=np.int64)
coefficients = np.ones(5)
kmoduli = np.ones(1)
ambient = np.array([4])

We can now initiate the PointGenerator

In [5]:
pg = PointGenerator(monomials, coefficients, kmoduli, ambient)

and generate some (100) points:

In [6]:
points = pg.generate_points(100)
points

array([[-0.26571881-9.17297455e-01j, -0.36345359+2.42049070e-01j,
         1.        +0.00000000e+00j, -0.59842131-1.43934436e-01j,
        -0.35605605-6.91997143e-01j],
       [ 1.        +5.55111512e-17j, -0.31804611-5.70832600e-01j,
         0.78152219+6.00384439e-01j,  0.15162194-1.69715014e-02j,
         0.31991803-3.70442042e-01j],
       [ 1.        -2.77555756e-17j, -0.23516908-2.60266458e-01j,
        -0.77837294+1.05176966e-01j,  0.81948808-4.76754593e-01j,
         0.66788379+2.93563540e-01j],
       [ 0.8244501 +5.64605799e-01j, -0.16831985-6.66244130e-01j,
         1.        -5.55111512e-17j,  0.28974831+6.27956749e-01j,
        -0.05495057-4.40566702e-02j],
       [-0.18569486+7.50333720e-01j,  0.24012511-3.46593059e-01j,
         1.        +5.55111512e-17j, -0.31459026+8.85538129e-01j,
        -0.45377699-5.09986505e-02j],
       [-0.62309363+5.81626089e-02j, -0.4679331 +6.79700462e-02j,
        -0.54795213-2.22345813e-02j,  1.        +0.00000000e+00j,
        -0.9677924

We see that the largest coordinate of each point is 1+0.j. The reason for that is two-fold:

1. The numerics are more stable when we work with points, which have affine coordinates in the range of $|x_i| < 1$.
2. The coordinate with 1+0.j already specifies the ambient space patch we are working in.

We can also check if these points are all really satisfying the hypersurface equation of the Calabi-Yau.

In [7]:
pg.cy_condition(points)

array([ 2.44249065e-15+5.55111512e-16j,  2.25514052e-16+4.03149736e-15j,
       -5.55111512e-17-3.94129174e-15j,  5.53415223e-15+2.23377072e-15j,
       -1.88044025e-15+1.87176663e-15j,  1.22124533e-15+1.85268467e-15j,
       -2.22044605e-16+9.85322934e-16j, -5.55111512e-16+2.77555756e-16j,
       -8.88178420e-16+5.55111512e-17j, -1.05818132e-15-1.47451495e-16j,
       -8.48930301e-16-8.58688121e-16j,  1.07147635e-15-3.20425110e-15j,
        3.74700271e-16-3.02839351e-15j,  8.02580658e-16-1.43307810e-16j,
        5.16513915e-16+2.09251019e-15j,  0.00000000e+00-1.11022302e-16j,
        4.88498131e-15-2.96984659e-15j,  2.22044605e-16-1.38777878e-17j,
        1.49880108e-14-1.13242749e-14j,  8.21565038e-15-5.88418203e-15j,
        2.22044605e-16+8.46545056e-16j, -8.88178420e-16-2.08166817e-15j,
       -1.35742112e-16-1.55431223e-15j, -7.49400542e-16-1.33226763e-15j,
        1.47104551e-15-7.56339436e-16j, -6.72205347e-18+1.08420217e-16j,
        2.63677968e-15-3.38618023e-15j,  0.00000000

what we are really interested in from the *PointGenerator* is a training set for our neural networks. Such a training set can be generated as follows:

In [8]:
help(pg.prepare_dataset)

Help on method prepare_dataset in module cymetric.pointgen.pointgen:

prepare_dataset(n_p, dirname, val_split=0.1, ltails=0, rtails=0) method of cymetric.pointgen.pointgen.PointGenerator instance
    Prepares training and validation data.
    
    Args:
        n_p (int): Number of points to generate.
        dirname (str): Directory name to save dataset in.
        val_split (float, optional): train-val split. Defaults to 0.1.
        ltails (float, optional): Percentage discarded on the left tail
            of weight distribution. Defaults to 0.
        rtails (float, optional): Percentage discarded on the right tail
            of weight distribution. Defaults to 0.
    
    Returns:
        int: 0



We specify the number of points and the directory name to save the file in. Note the file will always have the name *dataset.npz*.

In [9]:
dirname = 'fermat_pg'
n_p = 100000

and generate the dataset

In [10]:
pg.prepare_dataset(n_p, dirname)

0

We load the dataset with

In [11]:
data = np.load(os.path.join(dirname, 'dataset.npz'))

and study its content

In [12]:
for key in data:
    print(key, type(data[key]))

X_train <class 'numpy.ndarray'>
y_train <class 'numpy.ndarray'>
X_val <class 'numpy.ndarray'>
y_val <class 'numpy.ndarray'>
val_pullbacks <class 'numpy.ndarray'>


It contains training and validation data and the validation pullbacks. You might ask, what is written in the y_true values for validation and training data, given that we don't know the exact Ricci-flat metric. The 'y_train/val' arrays contain the integration weights and $\Omega \wedge \bar\Omega$ for each point. In principle, they can be used for any relevant pointwise information that could be needed during the training process.

In [13]:
weights = data['y_val'][:,0]
omega = data['y_val'][:,1]

we can also compute these values directly with the *PointGenerator*. Note, that the points in 'X_train/val' are floats, because our neural network will work with real values. We can recover the complex points as follows:

In [14]:
points = data['X_val'][:,0:pg.ncoords]+1.j*data['X_val'][:,pg.ncoords:]

and then compute the weights

In [15]:
weights2 = pg.point_weight(points)
np.allclose(weights, weights2)

True

and the holomorphic volume form

In [16]:
omega2 = pg.holomorphic_volume_form(points)
omega2 = omega2 * np.conj(omega2)
np.allclose(omega, omega2)

True

We will have to give information of the monomials and their derivatives to the tensorflow model. For this purpose we will create pickled dictionary denoted by *BASIS*.

In [17]:
pg.prepare_basis(dirname)

0

Let's have a look at the information stored in *basis.pickle*

In [18]:
with open(os.path.join(dirname, 'basis.pickle'), 'rb') as f:
    basis = pickle.load(f)
for key in basis:
    print(key)

DQDZB0
DQDZF0
QB0
QF0
NFOLD
AMBIENT
KMODULI
NHYPER


So in case you want to use your own PointGenerator with our TensorFlow models you will have to create a similar basis dictionary. Here we briefly describe what each of these keys stands for. In general *Q* denotes the defining hypersurface(s) with the final integer digit denoting the hypersurface index. *D* refers to derivatives, *Z* to the ambient space coordinates, *B* to a monomials basis and *F* for factors/coefficients for each monomial.

1. "DQDZB0": $\frac{\partial Q_0}{\partial z_i}$ monomials basis of derivatives of the first (and only for quintic) hypersurface w.r.t. ambient coordinates. 
2. "DQDZF0": $\frac{\partial Q_0}{\partial z_i}$ coefficients of the monomial basis.
3. "QB0": monomials basis for first hypersurface
4. "QF0": coefficients for monomial basis.
5. "NFOLD": CY dimension.
6. "AMBIENT": degrees of projective spaces making up the ambient space.
6. "KMODULI": kähler moduli corresponding to each projective factors. Note the CY needs to be favourable, otherwise you will have some superposition.
7. "NHYPER": number of hypersurfaces.

That pretty much sums up our introduction to the *PointGenerator* class, next we will implement the Fermat quintic in the *CICYPointGenerator*.

## CICYPointGenerator

The *CICYPointGenerator* and *ToricPointGenerator* come with the same functionality and routines as the *PointGenerator*. 

First we load from the cymetric package.

In [19]:
from cymetric.pointgen.pointgen_cicy import CICYPointGenerator

In contrast to the *PointGenerator* the *CICYPointGenerator* expects a list of monomials and coefficients. We reuse our previous monomials (and coefficients) with

In [20]:
pgcicy = CICYPointGenerator([monomials], [coefficients], kmoduli, ambient)

we again create a dataset

In [21]:
dirname = 'fermat_pgcicy'

In [22]:
pgcicy.prepare_dataset(n_p, dirname)

/home/robin/miniconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/robin/miniconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/robin/miniconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/robin/miniconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/robin/miniconda3/lib/pytho

0

as you might have realised, this took significantly longer than before. The reason for that is also related to all the warnings. The *CICYPointGenerator* utilises *scipy.optimize.fsolve* to find solutions on CICYs. *fsolve* only provides a single root of the defining hypersurfaces and requires more involved numerics, thus leading to worse performance (also in accuracy).

We again create a basis for the tensorflow models

In [23]:
pgcicy.prepare_basis(dirname)
with open(os.path.join(dirname, 'basis.pickle'), 'rb') as f:
    basis = pickle.load(f)
for key in basis:
    print(key)

DQDZB0
DQDZF0
QB0
QF0
NFOLD
AMBIENT
KMODULI
NHYPER


and see that the keys are identical to before.

## ToricPointGenerator

The *ToricPointGenerator* is somewhat special and requires additional input data generated from [SageMath](https://www.sagemath.org/). This *toric_data* can be straightforwardly generated in any sage kernel that has access to the *cymetric* package. In praxis only a single module is needed which can be found [here](../cymetric/sage/sagelib.py). 

The next cell won't work in your regular python notebook because it requires some sage routines for toric geometry. See [here](https://doc.sagemath.org/html/en/reference/schemes/sage/schemes/toric/variety.html) for more information about toric varieties and their implementation inn sage and [here](https://doc.sagemath.org/html/en/reference/discrete_geometry/sage/geometry/triangulation/point_configuration.html) for information about triangulations of PointCollections.

We begin by setting up the quintic vertices, which define the fan of the toric ambient variety. After initialising said variety we load the *prepare_toric_cy_data()* routine and generate the neccessary data for the *ToricPointGenerator* and the toric TensorFlow models.

In [24]:
#from cymetric.sage.sagelib import prepare_toric_cy_data
#import os as os
## Quintic vertices
#vertices = [
#    [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [-1, -1, -1, -1]
#]
#origin = [0 for _ in range(len(vertices[0]))]
#polytope = LatticePolytope(vertices)
#pConfig = PointConfiguration(polytope.points(), star=origin)
##set to topcom for more efficient triangulations
#pConfig.set_engine("TOPCOM")
## restrict to fine star regular.
#triangulations = pConfig.restrict_to_connected_triangulations()
#triangulations = triangulations.restrict_to_fine_triangulations()
#triangulations = triangulations.restrict_to_regular_triangulations()
#triangulations = triangulations.restrict_to_star_triangulations(origin)
##take first triangulation; build fan and TV
#triang = triangulations.triangulate()
#tv_fan = triang.fan()
#TV = ToricVariety(tv_fan)
#fname = os.path.join('fermat_pgtoric', 'toric_data.pickle')
#toric_data = prepare_toric_cy_data(TV, fname)

We are now in a position to go back to our regular python kernel or simply continue in Sage and load the *ToricPointGenerator*.

In [25]:
from cymetric.pointgen.pointgen_toric import ToricPointGenerator

Let's have a look at the information that has been written to *toric_data.pickle*.

In [26]:
dirname = 'fermat_pgtoric'
with open(os.path.join(dirname, 'toric_data.pickle'), 'rb') as f:
    toric_data = pickle.load(f)
for key in toric_data:
    print(key)
    print(toric_data[key])

dim_cy
3
vol_j_norm
5
coeff_aK
[(-0.2960611459754949+0.45586456886531984j), (-2.0881286576829687-0.03929679960298018j), (1.2830990967696805-1.2875193493048591j), (1.3825815487656659+0.053908971048372036j), (0.0363216119948742+0.9651917862914251j), (0.49241042799951684+0.2871084697021768j), (-2.0300416932070715-0.5375686724929758j), (0.341264022079822-0.9772090037570772j), (0.14182832791411604+0.20666837391395337j), (0.5159286452639563-0.2187621719220717j), (2.0135694399931228+1.517360231898885j), (-0.10898182279047733-0.3024577838145956j), (-0.4647914419189523-0.08799362843383562j), (2.0471158317038918+0.5887474822414696j), (0.034025746118910065+1.6728897679263075j), (0.2436302945160277-0.3714315452724199j), (0.6568460314592613-0.02513115884747583j), (-0.11145418092746642+0.9801645189317999j), (1.3996470611411447-1.4808746581187278j), (-0.21335172522709628+0.4870041095597406j), (-0.41712282178439347+1.2516254768452837j), (1.326394526533905+0.7807967198366524j), (-0.2350407327329405+1.0

We have the following keys:

1. "dim_cy": contains the dimension of the Calabi-Yau.
2. "vol_j_norm": information for normalisation of weights.
2. "coeff_aK": are generic complex coefficients in front of each of the defining hypersurface monomials, which you get from the Batyrev construction. Note: Those are by default complex valued as they represent a (redundant) description of the complex moduli.
3. "exp_aK": is the monomial basis for the defining equation.
4. "exps_sections": is a monomial basis for the sections of the kähler cone generators. This one will be needed to generate the integration weights and a Kähler metric in the same Kähler class as our Ricci-flat metric.
5. "patch_masks": are (boolean) coordinate masks for all the patches in the TV.
6. "glsm_charges": are the GLSM charges of the TV.
7. "triple": are the triple intersection numbers of the TV.

Having loaded the toric_data we can initiallise the ToricPointGenerator

In [27]:
pgtoric_gen = ToricPointGenerator(toric_data, kmoduli)

We now have a generic quintic with coefficients in front of all 125 monomials.

What if we only want the Fermat Quintic? We can override the information in "coeff_aK" and "exp_aK" with e.g.

In [28]:
toric_data["coeff_aK"] = coefficients
toric_data["exp_aK"] = monomials

and

In [29]:
pgtoric = ToricPointGenerator(toric_data, kmoduli)

we continue just as before with creating a dataset

In [30]:
pgtoric.prepare_dataset(n_p, dirname)

/home/robin/miniconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/robin/miniconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/robin/miniconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/robin/miniconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/robin/miniconda3/lib/pytho

/home/robin/miniconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/robin/miniconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/robin/miniconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/home/robin/miniconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/robin/miniconda

0

We note, that the data generation again took quite some time even though there should just be a single hypersurface. We also optimize w.r.t. to the sections coming from the Kähler generators and thus have more than a single hypersurface to consider.

We create the *BASIS* containing all information for the TensorFlow models:

In [31]:
pgtoric.prepare_basis(dirname)
with open(os.path.join(dirname, 'basis.pickle'), 'rb') as f:
    basis = pickle.load(f)
for key in basis:
    print(key)

DQDZB0
DQDZF0
QB0
QF0
NFOLD
AMBIENT
KMODULI
NHYPER


In the next notebook we will load the *basis.pickle* and *toric_data.pickle* for the TensorFlow model. The information in *basis.pickle* will override what is written in *toric_data.pickle* so that the models know we want to work with the Fermat and not some generic quintic.

We can also check if the points are all on the fermat:

In [32]:
data = np.load(os.path.join(dirname, 'dataset.npz'))
points = data['X_val'][:,0:pg.ncoords]+1.j*data['X_val'][:,pg.ncoords:]

using the regular *PointGenerator*

In [33]:
np.sum(np.isclose(np.abs(pg.cy_condition(points)), 0))/len(points)

1.0

the toric

In [34]:
np.sum(np.isclose(np.abs(pgtoric.cy_condition(points)), 0))/len(points)

1.0

and the cicy

In [35]:
np.sum(np.isclose(np.abs(pgcicy.cy_condition(points)), 0))/len(points)

1.0